In [6]:
import numpy as np
import matplotlib.pyplot as plt
import re


In [3]:
def extract_mcmc_parameters(file_path):
    # Read the file content
    with open(file_path, "r") as file:
        data = file.read()
    
    # Extract key-value pairs using regular expressions
    results = re.findall(r'(\S+)\s*=\s*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)\s*(\+\d*\.?\d+(?:[eE][-+]?\d+)?)?\s*(\-\d*\.?\d+(?:[eE][-+]?\d+)?)?', data)
    
    # Store results in a dictionary
    parameters = {}
    for result in results:
        key = result[0]
        value = float(result[1])
        plus_err = float(result[2]) if result[2] else None
        minus_err = float(result[3]) if result[3] else None
        parameters[key] = {
            "value": value,
            "plus_err": plus_err,
            "minus_err": minus_err
        }
    
    return parameters

In [11]:
# Select models tested
e1 = ['exp', 'exp+linear', 'exp+polynomial', 'polynomial', 'double_exp', 'exp_2nd_order_centroid', 'exp+polynomial_2nd_order_centroid', 'exp+linear_2nd_order_centroid', 'polynomial_2nd_order_centroid', 'double_exp_2nd_order_centroid']
e2 =  ['exp', 'polynomial', 'linear+polynomial', 'linear+polynomial_2nd_order_centroid', 'exp_2nd_order_centroid','polynomial_2nd_order_centroid']
e3 = ['exp','polynomial', 'linear+polynomial', 'polynomial_2nd_order_centroid']

parameters_e1 = {model: [] for model in e1}
parameters_e2 = {model: [] for model in e2}
parameters_e3 = {model: [] for model in e3}

parameters = [parameters_e1, parameters_e2, parameters_e3]
i = 1
for params in parameters:
    print('Eclipse', i)
    for model in params:
        file = 'MCMC_results/ecl'+str(i)+'_' + model + '_test_results.txt'
        params[model] = extract_mcmc_parameters(file)
        print(model, params[model])
    i += 1


Eclipse 1
exp {'t_s': {'value': 2.3547243614739815, 'plus_err': 0.045854173692250644, 'minus_err': -0.0527589061881546}, 'fp': {'value': 0.00032398707243913577, 'plus_err': 5.360428359960632e-05, 'minus_err': -5.410135642869773e-05}, 'c_1': {'value': 0.0034604447608135407, 'plus_err': 9.410675957610823e-05, 'minus_err': -9.73657101093618e-05}, 'c_2': {'value': 2.135798552069624, 'plus_err': 0.06718036619464707, 'minus_err': -0.07133297468438293}, 'c_3': {'value': 0.9993458094628047, 'plus_err': 4.3435121287571654e-05, 'minus_err': -4.368229554829739e-05}, 'c_4': {'value': 0.0008097972210384756, 'plus_err': 1.6361233582886324e-05, 'minus_err': -1.581573278333615e-05}, 'RMS': {'value': 810.553281638424, 'plus_err': None, 'minus_err': None}, 'chi2': {'value': 1261.803615472046, 'plus_err': None, 'minus_err': None}, 'BIC': {'value': -16634.554431339307, 'plus_err': None, 'minus_err': None}}
exp+linear {'t_s': {'value': 2.3498684189533745, 'plus_err': 0.01386041209878508, 'minus_err': -0.01

In [22]:
# get fp averages for each eclipses
e1_fp_dict =  {}
e1_fp_errp_dict = {}
e1_fp_errm_dict = {}

e2_fp_dict =  {}
e2_fp_errp_dict = {}
e2_fp_errm_dict = {}

e3_fp_dict =  {}
e3_fp_errp_dict = {}
e3_fp_errm_dict = {}

for model in e1:
    e1_fp_dict[model], e1_fp_errp_dict[model], e1_fp_errm_dict[model] = parameters_e1[model]['fp']['value'], parameters_e1[model]['fp']['plus_err'], parameters_e1[model]['fp']['minus_err']
for model in e2:
    e2_fp_dict[model], e2_fp_errp_dict[model], e2_fp_errm_dict[model] = parameters_e2[model]['fp']['value'], parameters_e2[model]['fp']['plus_err'], parameters_e2[model]['fp']['minus_err']    
for model in e3:
    e3_fp_dict[model], e3_fp_errp_dict[model], e3_fp_errm_dict[model] = parameters_e3[model]['fp']['value'], parameters_e3[model]['fp']['plus_err'], parameters_e3[model]['fp']['minus_err']

In [45]:
weight_e1 = []
weight_e2 = []
weight_e3 = []

fp_e1 = []
fp_e2 = []
fp_e3 = []

for model in e1:
    weight_e1.append(e1_fp_errp_dict[model] + np.abs(e1_fp_errm_dict[model]))
    fp_e1.append(e1_fp_dict[model])
for model in e2:
    weight_e2.append(e2_fp_errp_dict[model] + np.abs(e2_fp_errm_dict[model]))
    fp_e2.append(e2_fp_dict[model])
for model in e3:
    weight_e3.append(e3_fp_errp_dict[model] + np.abs(e3_fp_errm_dict[model]))
    fp_e3.append(e3_fp_dict[model])

# weighted average
weight_e1 = np.array(weight_e1)
p_e1 = np.sum(fp_e1/(weight_e1**2))/np.sum(1/(weight_e1**2))
print('Weighted average of fp for e1:', p_e1*1e6)

weight_e2 = np.array(weight_e2)
p_e2 = np.sum(fp_e2/(weight_e2**2))/np.sum(1/(weight_e2**2))
print('Weighted average of fp for e2:', p_e2*1e6)

weight_e3 = np.array(weight_e3)
p_e3 = np.sum(fp_e3/(weight_e3**2))/np.sum(1/(weight_e3**2))
print('Weighted average of fp for e3:', p_e3*1e6)


Weighted average of fp for e1: 351.7348233707502
Weighted average of fp for e2: 179.79343209180328
Weighted average of fp for e3: 238.1908985634695
